# Ozone response in a model with and without interactive stratospheric chemistry
## Emma Axebrink
emma.axebrink@nuclear.lu.se

#### eScience Tools in Climate Science: Linking Observations with Modelling 

Assistant: Zhihong Zhuo <br>
zhihong.zhuo@geo.uio.no <br>
14 November 2022

# Abstract

# Introduction

- Why am i studying this topic
To run a model can cost a lot of computational power and therefore it is not always desirable to include all calculations in a model. If the interest of a variable lies at the surface, a model without interactiv stratospheric chemistry might be preferred since the computation in recuced compared to a model with interactive stratospheric chemistry. The downside of using a smaller model might however be that some variables are not responding as expected. This work aims to investigate this potential downside. More specifically; investigate the response to a volcanic eruption by looking at the ozone, AOD (Aerosol Optical Thickness) and temperature of two models. One with interactive stratospheric chemistry and one without.

- What is done in this notebook?

- Some background (on models and observations) and underlying theory (check for 2-3 papers with assistants and teachers if needed)
- Present equations if needed (Latex)
In this work the ozone in dobson unit is calculate from mol/mol with the following equations:


O3_mmm = O3_vmm*(48.0/28.94)

g=9.81
Plevi = ds.plev

dp = np.empty(shape=O3_vmm.shape)

dpa=xr.DataArray(dp,coords=ds.o3.coords,dims=ds.o3.dims)

for i in range(1,Plevi.shape[0]):
    dpa[dict(plev=i-1)]=-(Plevi[i]-Plevi[i-1])
    
O3_t=O3_mmm*dpa/g

totO3=O3_t.sum(dim='plev')

totO3DU = totO3/2.1415e-5


## Research questions

- How does ozone respond in a model with and without interactive stratospheric chemistry?
- Is the ozone response to volcanic eruptions amplified in the polar region?
- How does temperature respond in a model with and without interactive stratospheric chemistry? 
- How does AOD (Aerosol Optical Thickness) respond in a model with and without interactive stratospheric chemistry?


# Method

## Packages used

In [1]:
import xarray as xr
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from dask.distributed import Client
import pprint
# My funcions
#%run /home/jovyan/escience-2022/Tjaernoe2022-group2/Emma/func.ipynb
#%run /home/jovyan/escience-2022/Tjaernoe2022-group2/Emma/CMIP6_func.ipynb
from eScienceCourse_2022_EmmaAxebrink_functions import *
#from func import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Datasets 
**CMIP6 models**
CMIP6 is the sixth generation of the Coupeld Model Intercomparison Project, consisting of multiple models and a total of around one hundred runs.
In this report three historical runs of CESM2 CAM and CESM2 WACCM, six in total, have been chosen to investigate a model with and without interactive stratospheric chemistry. CAM does not have interactive stratospheric chemistry, i.e CAM utilise a prescribed stratosphere which is obtained by calculating the 5-day zonal-mean for each year over the years 1850-2014 [[1]]CMIP6 models 
CMIP 6 is the sixth generation of the Coupeld Model Intercomparison Project, consisting of multiple models and a total of around one hundred runs. In this work three historical runs of CESM2 CAM and three historical runs of CESM2 WACCM, six in total, have been chosen to investigate a model with and without interactive stratospheric chemistry. CAM does not have interactive stratospheric chemistry, i.e CAM utilise a prescribed stratosphere which is obtained by calculating the 5-day zonal-mean for each year over the years 1850-2014 [[1]].
The three runs for each model have the member id r1i1p1f1 ,r2i1p1f1 and r3i1p1f1. 
The variables investigated were o3, od550aer and ta, i.e. ozone, ambient aerosol optical thickness at 550nm and temperature. The time range for these two models are monthly averages from 1850 to 2014 but only two subsets of eleven years have been used, namely January 1986 to December 1996. ################## **what else?!** ##################



[1]: https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2019MS001916
*For the CESM2(CAM6) historical simulations, 5-day zonal-mean values were calculated for each year over the 1850–2014 historical period from the average of the three CESM2(WACCM6) historical ensemble members. The 5-day frequency allows resolution of the impacts of major volcanic eruptions, as well as the annual development of polar ozone loss (Neely et al., 2014). The averaging of three CESM2(WACCM6) historical ensemble members is especially important to the evolution of stratospheric aerosol resulting from volcanic eruptions, which can be particularly sensitive to the circulation present at the time of the eruption*

Since the datasets are large and take some time to load Dask is used to speed the process up by parallel computing

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 62.81 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35279,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 62.81 GiB
Comm: tcp://127.0.0.1:46693,Total threads: 4
Dashboard: http://127.0.0.1:33915/status,Memory: 15.70 GiB
Nanny: tcp://127.0.0.1:38415,


The data for both CAM and WACCM are located in the CMIP6 PANGEO catalogue. 

In [5]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
# Search the catalogue for ozone data 
waccm_o3 = col.search(source_id=['CESM2-WACCM'], experiment_id=['historical'], table_id=['Amon'], variable_id=['o3'], member_id=['r1i1p1f1','r2i1p1f1','r3i1p1f1'])
cam_o3 = col.search(source_id=['CESM2'], experiment_id=['historical'], table_id=['Amon'], variable_id=['o3'], member_id=['r1i1p1f1','r2i1p1f1','r3i1p1f1'])

# Search the catalogue for AOD data 
waccm_aod = col.search(source_id=['CESM2-WACCM'], experiment_id=['historical'], table_id=['AERmon'], variable_id=['od550aer'], member_id=['r1i1p1f1','r2i1p1f1','r3i1p1f1'])
cam_aod = col.search(source_id=['CESM2'], experiment_id=['historical'], table_id=['AERmon'], variable_id=['od550aer'], member_id=['r1i1p1f1','r2i1p1f1','r3i1p1f1'])

#Search the catalogue for temperature data 
waccm_t = col.search(source_id=['CESM2-WACCM'], experiment_id=['historical'], table_id=['Amon'], variable_id=['ta'], member_id=['r1i1p1f1','r2i1p1f1','r3i1p1f1'])
cam_t = col.search(source_id=['CESM2'], experiment_id=['historical'], table_id=['Amon'], variable_id=['ta'], member_id=['r1i1p1f1','r2i1p1f1','r3i1p1f1'])

In [6]:
dict_waccm_o3 = waccm_o3.to_dataset_dict(zarr_kwargs={'use_cftime':True}, cdf_kwargs={"chunks": {"time": 40}})
dict_cam_o3 = cam_o3.to_dataset_dict(zarr_kwargs={'use_cftime':True},cdf_kwargs={"chunks": {"time": 4}})

dict_waccm_aod = waccm_aod.to_dataset_dict(zarr_kwargs={'use_cftime':True}, cdf_kwargs={"chunks": {"time": 40}})
dict_cam_aod = cam_aod.to_dataset_dict(zarr_kwargs={'use_cftime':True},cdf_kwargs={"chunks": {"time": 4}})

dict_waccm_t = waccm_t.to_dataset_dict(zarr_kwargs={'use_cftime':True}, cdf_kwargs={"chunks": {"time": 40}})
dict_cam_t = cam_t.to_dataset_dict(zarr_kwargs={'use_cftime':True},cdf_kwargs={"chunks": {"time": 4}})



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [9]:
print(list(dict_waccm_o3.keys()))
print(list(dict_cam_o3.keys()))
waccm_o3 = dict_waccm_o3['CMIP.NCAR.CESM2-WACCM.historical.Amon.gn']
cam_o3 = dict_cam_o3['CMIP.NCAR.CESM2.historical.Amon.gn']

print(list(dict_waccm_aod.keys()))
print(list(dict_cam_aod.keys()))
waccm_aod = dict_waccm_aod['CMIP.NCAR.CESM2-WACCM.historical.AERmon.gn']
cam_aod = dict_cam_aod['CMIP.NCAR.CESM2.historical.AERmon.gn']

print(list(dict_waccm_t.keys()))
print(list(dict_cam_t.keys()))
waccm_t = dict_waccm_t['CMIP.NCAR.CESM2-WACCM.historical.Amon.gn']
cam_t = dict_cam_t['CMIP.NCAR.CESM2.historical.Amon.gn']

['CMIP.NCAR.CESM2-WACCM.historical.Amon.gn']
['CMIP.NCAR.CESM2.historical.Amon.gn']
['CMIP.NCAR.CESM2-WACCM.historical.AERmon.gn']
['CMIP.NCAR.CESM2.historical.AERmon.gn']
['CMIP.NCAR.CESM2-WACCM.historical.Amon.gn']
['CMIP.NCAR.CESM2.historical.Amon.gn']


To easier handle the datasets merge the variables for each dataset.

In [20]:
waccm = waccm_o3.merge(waccm_t,compat='override').merge(waccm_aod,compat='override')
cam = cam_o3.merge(cam_t,compat='override').merge(cam_aod,compat='override')

## Analysis method

To reuce the size of the datasets a specific time range is selected. In this work the eruption of Mount Pinatubo, January 1986 to December 1996 is selected. This time span will allow for calculations of the anomaly by taking the mean of 5 years prior the eruption, i.e the climatology, and substracting that from the remaining 6 years to get rid of any seasonaly variations.

In [25]:
# Select the starting year 5 years prior to the eruption
start_pina = cftime.DatetimeNoLeap(1986,1,15)
end_pina = cftime.DatetimeNoLeap(1996,12,15)

# WACCM
waccm_1986_1996 = waccm.sel(time=slice(start_pina, end_pina))
# CAM
cam_1986_1996 = cam.sel(time=slice(start_pina, end_pina))

Before calculating the anomaly for the three variables the ozone variable need to be converted from mol/mol to dobson units (DU). This is done by using the function *calculate_total_ozone_p* located in the eScienceCourse_2022_EmmaAxebrink_functions.py file. This function will create a new variable called *totO3*.

In [29]:
waccm_1986_1996 = calculate_total_ozone_p(waccm_1986_1996)
cam_1986_1996 = calculate_total_ozone_p(cam_1986_1996)

In [32]:
# Yearly
year_waccm_anom_1986_1996_o3 = calc_yearly_anomaly(waccm_1986_1996.totO3,1991)
year_cam_anom_1986_1996_o3 = calc_yearly_anomaly(cam_1986_1996.totO3,1991)

year_waccm_anom_1986_1996_aod = calc_yearly_anomaly(waccm_1986_1996.od550aer,1991)
year_cam_anom_1986_1996_aod = calc_yearly_anomaly(cam_1986_1996.od550aer,1991)

year_waccm_anom_1986_1996_t = calc_yearly_anomaly(waccm_1986_1996.ta,1991)
year_cam_anom_1986_1996_t = calc_yearly_anomaly(cam_1986_1996.ta,1991)

# Monthly
mon_waccm_anom_1986_1996_o3 = calc_monthly_anomaly(waccm_1986_1996.totO3,1991)
mon_cam_anom_1986_1996_o3 = calc_monthly_anomaly(cam_1986_1996.totO3,1991)

mon_waccm_anom_1986_1996_aod = calc_monthly_anomaly(waccm_1986_1996.od550aer,1991)
mon_cam_anom_1986_1996_aod = calc_monthly_anomaly(cam_1986_1996.od550aer,1991)

mon_waccm_anom_1986_1996_t = calc_monthly_anomaly(waccm_1986_1996.ta,1991)
mon_cam_anom_1986_1996_t = calc_monthly_anomaly(cam_1986_1996.ta,1991)

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 11 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 11 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 11 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 11 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 11 times more chunks
  return self.array[key]
/srv/conda

To compare the two models the three members in CAM and the three members WACCM are plotted in the same subplot using the function *plot2_year* or *plot2_mon* the 2 in the name indicated that the plot will contain 2 subplots which will be the northern hemisphere > 60N and the southern hemisphere < 60S. The function *plot3_year* will contain an additional subplot with the latitude 60S ~ 60N

# Results

# Discussion and outlook

# Conclusions

# References

[1]: https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2019MS001916
\[1]\: https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2019MS001916

# Acknowledgments

# Supplementary materials